In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('max.column', None)

import plotly.express as px

In [173]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler 

# import various functions from sklearn
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

# import the functions for visualizing the decision tree
# import pydotplus
# from IPython.display import Image  

In [174]:
# read dataset

df = pd.read_csv('Lending Club 2016_2018.csv')

In [175]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,tot_coll_amt,tot_cur_bal,issue_d,chargeoff_within_12_mths,acc_now_delinq,mort_acc,mths_since_recent_bc_dlq,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,loan_status
0,130956066,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,9 years,RENT,52000.0,Source Verified,major_purchase,WA,0.58,0.0,0.0,26.0,NaN,7.0,0.0,141.0,0.5,30.0,w,0.0,NaN,0.0,150592.0,Mar-2018,0.0,0.0,4.0,NaN,0.0,96.7,0.0,191216.0,0
1,130968727,5000.0,5000.0,5000.0,36 months,11.98,166.03,B,10+ years,OWN,55000.0,Not Verified,other,GA,14.18,0.0,0.0,74.0,82.0,14.0,1.0,11449.0,33.9,24.0,w,0.0,74.0,0.0,28880.0,Mar-2018,0.0,0.0,0.0,NaN,1.0,95.7,1.0,61551.0,0
2,130910225,7000.0,7000.0,7000.0,36 months,11.98,232.44,B,< 1 year,MORTGAGE,40000.0,Verified,home_improvement,TX,20.25,0.0,0.0,60.0,NaN,13.0,0.0,5004.0,36.0,29.0,w,0.0,60.0,0.0,131726.0,Mar-2018,0.0,0.0,0.0,64.0,3.0,89.7,0.0,132817.0,0
3,130966492,30000.0,30000.0,30000.0,36 months,21.85,1143.39,D,10+ years,OWN,57000.0,Verified,debt_consolidation,FL,27.58,0.0,1.0,68.0,NaN,11.0,0.0,29222.0,53.2,26.0,w,0.0,68.0,0.0,157566.0,Mar-2018,0.0,0.0,2.0,NaN,1.0,96.0,0.0,188780.0,0
4,130942737,21000.0,21000.0,21000.0,60 months,20.39,560.94,D,10+ years,OWN,85000.0,Source Verified,house,NY,15.76,1.0,0.0,2.0,NaN,15.0,0.0,14591.0,34.2,27.0,w,0.0,NaN,0.0,128270.0,Mar-2018,0.0,0.0,3.0,NaN,0.0,92.6,0.0,172433.0,0


In [176]:
df.shape

(518706, 39)

In [177]:
df_copy = df.copy()

In [178]:
df_copy.drop(['id','addr_state','issue_d','funded_amnt','funded_amnt_inv'],axis=1,inplace=True)

In [179]:
for i in ['loan_amnt', 'int_rate',
       'installment', 'annual_inc', 'dti', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'revol_util', 'total_acc']:
    Q1 = df_copy[i].quantile(0.25)
    Q3 = df_copy[i].quantile(0.75)

    IQR = Q3-Q1
    df_temp = df_copy[(df_copy[i] > (Q1 - 1.5*IQR)) & (df_copy[i] < (Q3 + 1.5*IQR))]

In [180]:
for i in ['collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'tot_cur_bal', 'chargeoff_within_12_mths',
       'acc_now_delinq', 'mort_acc', 'mths_since_recent_bc_dlq',
       'num_accts_ever_120_pd','tot_hi_cred_lim']:
    Q1 = df_temp[i].quantile(0.25)
    Q3 = df_temp[i].quantile(0.75)

    IQR = Q3-Q1
    df_no_out = df_temp[(df_temp[i] > (Q1 - 1.5*IQR)) & (df_temp[i] < (Q3 + 1.5*IQR))]

In [181]:
df_no_out.dropna(subset = ['emp_length','dti','revol_util','pct_tl_nvr_dlq','inq_last_6mths'], inplace = True)


In [182]:
cols = ['mths_since_last_major_derog','mths_since_recent_bc_dlq','mths_since_last_record','mths_since_last_delinq']
for i in cols:
    df_no_out[i] = pd.cut(df[i], 
           bins = np.arange(0,262,36), 
           labels=['0-3','4-6','7-9','10-12','13-15','16-18','18+'])

    df_no_out[i] = df_no_out[i].astype('object')

    df_no_out[i].replace(np.nan, 'None', inplace = True)

In [183]:
df_no_out.head()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,tot_coll_amt,tot_cur_bal,chargeoff_within_12_mths,acc_now_delinq,mort_acc,mths_since_recent_bc_dlq,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,loan_status
0,3000.0,36 months,7.34,93.10,A,9 years,RENT,52000.0,Source Verified,major_purchase,0.58,0.0,0.0,0-3,None,7.0,0.0,141.0,0.5,30.0,w,0.0,None,0.0,150592.0,0.0,0.0,4.0,None,0.0,96.7,0.0,191216.0,0
1,5000.0,36 months,11.98,166.03,B,10+ years,OWN,55000.0,Not Verified,other,14.18,0.0,0.0,7-9,7-9,14.0,1.0,11449.0,33.9,24.0,w,0.0,7-9,0.0,28880.0,0.0,0.0,0.0,None,1.0,95.7,1.0,61551.0,0
2,7000.0,36 months,11.98,232.44,B,< 1 year,MORTGAGE,40000.0,Verified,home_improvement,20.25,0.0,0.0,4-6,None,13.0,0.0,5004.0,36.0,29.0,w,0.0,4-6,0.0,131726.0,0.0,0.0,0.0,4-6,3.0,89.7,0.0,132817.0,0
3,30000.0,36 months,21.85,1143.39,D,10+ years,OWN,57000.0,Verified,debt_consolidation,27.58,0.0,1.0,4-6,None,11.0,0.0,29222.0,53.2,26.0,w,0.0,4-6,0.0,157566.0,0.0,0.0,2.0,None,1.0,96.0,0.0,188780.0,0
4,21000.0,60 months,20.39,560.94,D,10+ years,OWN,85000.0,Source Verified,house,15.76,1.0,0.0,0-3,None,15.0,0.0,14591.0,34.2,27.0,w,0.0,None,0.0,128270.0,0.0,0.0,3.0,None,0.0,92.6,0.0,172433.0,0


In [184]:
df_no_out.select_dtypes(include='object').head()

,term,grade,emp_length,home_ownership,verification_status,purpose,mths_since_last_delinq,mths_since_last_record,initial_list_status,mths_since_last_major_derog,mths_since_recent_bc_dlq
0,36 months,A,9 years,RENT,Source Verified,major_purchase,0-3,None,w,None,None
1,36 months,B,10+ years,OWN,Not Verified,other,7-9,7-9,w,7-9,None
2,36 months,B,< 1 year,MORTGAGE,Verified,home_improvement,4-6,None,w,4-6,4-6
3,36 months,D,10+ years,OWN,Verified,debt_consolidation,4-6,None,w,4-6,None
4,60 months,D,10+ years,OWN,Source Verified,house,0-3,None,w,None,None


In [185]:
term_values = {' 36 months': 36, ' 60 months': 60}
df_no_out['term'] = df_no_out.term.map(term_values)

In [186]:
LE = LabelEncoder()
df_no_out['emp_length']= LE.fit_transform(df_no_out['emp_length'])

In [187]:
dummies = ['grade','home_ownership','verification_status','purpose','initial_list_status','mths_since_last_delinq','mths_since_last_record','mths_since_last_major_derog','mths_since_recent_bc_dlq']

In [188]:
data = pd.get_dummies(df_no_out, columns=dummies, drop_first=True)

In [189]:
data.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,chargeoff_within_12_mths,acc_now_delinq,mort_acc,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,loan_status,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,mths_since_last_delinq_10-12,mths_since_last_delinq_13-15,mths_since_last_delinq_16-18,mths_since_last_delinq_18+,mths_since_last_delinq_4-6,mths_since_last_delinq_7-9,mths_since_last_delinq_None,mths_since_last_record_10-12,mths_since_last_record_4-6,mths_since_last_record_7-9,mths_since_last_record_None,mths_since_last_major_derog_10-12,mths_since_last_major_derog_13-15,mths_since_last_major_derog_16-18,mths_since_last_major_derog_18+,mths_since_last_major_derog_4-6,mths_since_last_major_derog_7-9,mths_since_last_major_derog_None,mths_since_recent_bc_dlq_10-12,mths_since_recent_bc_dlq_13-15,mths_since_recent_bc_dlq_16-18,mths_since_recent_bc_dlq_4-6,mths_since_recent_bc_dlq_7-9,mths_since_recent_bc_dlq_None
0,3000.0,36,7.34,93.10,9,52000.0,0.58,0.0,0.0,7.0,0.0,141.0,0.5,30.0,0.0,0.0,150592.0,0.0,0.0,4.0,0.0,96.7,0.0,191216.0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
1,5000.0,36,11.98,166.03,1,55000.0,14.18,0.0,0.0,14.0,1.0,11449.0,33.9,24.0,0.0,0.0,28880.0,0.0,0.0,0.0,1.0,95.7,1.0,61551.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,7000.0,36,11.98,232.44,10,40000.0,20.25,0.0,0.0,13.0,0.0,5004.0,36.0,29.0,0.0,0.0,131726.0,0.0,0.0,0.0,3.0,89.7,0.0,132817.0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
3,30000.0,36,21.85,1143.39,1,57000.0,27.58,0.0,1.0,11.0,0.0,29222.0,53.2,26.0,0.0,0.0,157566.0,0.0,0.0,2.0,1.0,96.0,0.0,188780.0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
4,21000.0,60,20.39,560.94,1,85000.0,15.76,1.0,0.0,15.0,0.0,14591.0,34.2,27.0,0.0,0.0,128270.0,0.0,0.0,3.0,0.0,92.6,0.0,172433.0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1


In [190]:
data.shape

(453227, 74)

In [191]:
df_target = data['loan_status']
X = data.drop('loan_status', axis = 1)

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, df_target, random_state = 10, test_size = 0.2)

print('X_train', X_train.shape)
print('y_train', y_train.shape)

print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (362581, 73)
y_train (362581,)
X_test (90646, 73)
y_test (90646,)


In [193]:
X_train.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,chargeoff_within_12_mths,acc_now_delinq,mort_acc,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,mths_since_last_delinq_10-12,mths_since_last_delinq_13-15,mths_since_last_delinq_16-18,mths_since_last_delinq_18+,mths_since_last_delinq_4-6,mths_since_last_delinq_7-9,mths_since_last_delinq_None,mths_since_last_record_10-12,mths_since_last_record_4-6,mths_since_last_record_7-9,mths_since_last_record_None,mths_since_last_major_derog_10-12,mths_since_last_major_derog_13-15,mths_since_last_major_derog_16-18,mths_since_last_major_derog_18+,mths_since_last_major_derog_4-6,mths_since_last_major_derog_7-9,mths_since_last_major_derog_None,mths_since_recent_bc_dlq_10-12,mths_since_recent_bc_dlq_13-15,mths_since_recent_bc_dlq_16-18,mths_since_recent_bc_dlq_4-6,mths_since_recent_bc_dlq_7-9,mths_since_recent_bc_dlq_None
183098,12000.0,60,28.69,378.65,0,40600.0,15.22,2.0,1.0,34.0,0.0,1171.0,32.5,43.0,1.0,1984.0,272850.0,0.0,0.0,0.0,0.0,95.3,0.0,243511.0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
513511,7600.0,36,13.49,257.88,0,45000.0,23.79,0.0,0.0,16.0,0.0,8654.0,69.2,19.0,0.0,0.0,58211.0,0.0,0.0,0.0,0.0,94.7,0.0,64419.0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
132065,12000.0,36,10.47,389.86,3,60000.0,31.92,0.0,1.0,25.0,1.0,14335.0,20.3,46.0,0.0,0.0,192142.0,0.0,0.0,1.0,0.0,100.0,1.0,270811.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
276723,12000.0,60,13.44,275.75,1,65000.0,20.61,0.0,0.0,9.0,0.0,16639.0,79.2,19.0,0.0,0.0,40741.0,0.0,0.0,0.0,0.0,100.0,0.0,58334.0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
16641,35000.0,36,10.91,1144.37,1,145000.0,14.34,0.0,0.0,12.0,0.0,27002.0,55.9,24.0,0.0,0.0,81492.0,0.0,0.0,0.0,0.0,100.0,0.0,130551.0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1


In [197]:
rf_classification = RandomForestClassifier(n_estimators = 10, random_state = 10)

# use fit() to fit the model on the train set
rf_model = rf_classification.fit(X_train, y_train)

In [198]:
y_train_pred = rf_model.predict(X_train)

print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99    282239
           1       1.00      0.90      0.95     80342

    accuracy                           0.98    362581
   macro avg       0.99      0.95      0.97    362581
weighted avg       0.98      0.98      0.98    362581



In [199]:
y_test_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.79      0.96      0.87     70558
           1       0.45      0.12      0.19     20088

    accuracy                           0.77     90646
   macro avg       0.62      0.54      0.53     90646
weighted avg       0.72      0.77      0.72     90646

